In [32]:
%load_ext autoreload
%autoreload 2
import numpy as np
from os.path import join as oj
out_dir = '../../results/nlp'
os.makedirs(out_dir, exist_ok=True)
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import pandas as pd
import pickle as pkl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# look at smth with attention...

In [ ]:
from pytorch_transformers import BertTokenizer, BertForMaskedLM, BertForSequenceClassification, BertConfig
from util import *
device = 'cpu'

default = 'bert-base-uncased'
mdir = '/scratch/users/vision/chandan/pacmed/glue/SST-2-3epoch' # '/scratch/users/vision/chandan/pacmed/glue/SST-2-middle/'

tokenizer = BertTokenizer.from_pretrained(mdir)
clf = BertForSequenceClassification.from_pretrained(mdir).eval().to(device)
masked_predictor = BertForMaskedLM.from_pretrained(default).eval().to(device)

In [ ]:
s = 'good movie'
with torch.no_grad():
    print('encoded', tokenizer.convert_ids_to_tokens(tokenizer.encode(s)))
    input_ids = torch.tensor(tokenizer.encode(s), device=device).unsqueeze(0)  # Batch size 1
    pred = clf(input_ids)[0].detach().cpu().softmax(dim=1).numpy().flatten()

In [ ]:
# integrated gradients
def integrated_gradients(inputs, model, target_label_idx, predict_and_gradients, baseline, steps=50, cuda=False):
    if baseline is None:
        baseline = 0 * inputs 
    # scale inputs and compute gradients
    scaled_inputs = [baseline + (float(i) / steps) * (inputs - baseline) for i in range(0, steps + 1)]
    grads, _ = predict_and_gradients(scaled_inputs, model, target_label_idx, cuda)
    avg_grads = np.average(grads[:-1], axis=0)
    avg_grads = np.transpose(avg_grads, (1, 2, 0))
    integrated_grad = (inputs - baseline) * avg_grads
    return integrated_grad

def predict_and_gradients(inputs, model, target_label_idx, cuda=False):
    # do the pre-processing
    predict_idx = None
    gradients = []
    print('inputs', inputs)
    for inp in inputs:
        # inp = pre_processing(input, cuda)
        output = model(inp)[0]
        output = F.softmax(output, dim=1)
        if target_label_idx is None:
            target_label_idx = torch.argmax(output, 1).item()
        index = np.ones((output.size()[0], 1)) * target_label_idx
        index = torch.tensor(index, dtype=torch.int64)
        if cuda:
            index = index.cuda()
        output = output.gather(1, index)
        
        # clear grad
        model.zero_grad()
        output.backward()
        print('inp', inp)
        print('inp.grad', inp.grad)
        gradient = inp.grad.detach().cpu().numpy()[0]
        gradients.append(gradient)
        
    gradients = np.array(gradients)
    return gradients, target_label_idx

integrated_gradients(input_ids, clf, 1, predict_and_gradients, 0)

# look at smth with attention

In [9]:
clf = BertForSequenceClassification.from_pretrained(mdir, output_attentions=True).eval().to(device)

s = 'good movie'
with torch.no_grad():
    # print('encoded', tokenizer.convert_ids_to_tokens(tokenizer.encode(s)))
    input_ids = torch.tensor(tokenizer.encode(s), device=device).unsqueeze(0)  # Batch size 1
    pred = clf(input_ids)#[0].detach().cpu().softmax(dim=1).numpy().flatten()
    all_attentions = pred[1]
    pred = pred[0].detach().cpu().softmax(dim=1).numpy().flatten()